## consider the d dimensional model

In [1]:
import numpy as np
from mip import Model, xsum, minimize, maximize, OptimizationStatus
from numpy.polynomial.polynomial import polyfit
import matplotlib.pyplot as plt
from numpy.linalg import inv
import time

EXP_TIMES = 500
TIME_LEVELS = 20
SAMPLE_AMOUNT = 1000

collection = np.zeros(EXP_TIMES)

start_time = time.time()
batch = 0

for exp in range(EXP_TIMES):
    # generate
    X1 = np.random.uniform(-1,1,SAMPLE_AMOUNT)
    X2 = np.random.uniform(-1,1,SAMPLE_AMOUNT)
    # X3 = np.random.uniform(-1,1,SAMPLE_AMOUNT)
    # X_2and3 = np.column_stack((X2, X3))
    
    EPSI1 = np.random.uniform(0,1,SAMPLE_AMOUNT)
    EPSI2 = np.random.uniform(0,1,SAMPLE_AMOUNT)
    
    Y1 = X1 + 2 + EPSI1 - EPSI2
    Y2 = X1 + 3 + EPSI1 + EPSI2
    
    # phi(x) = x^2
    A = 4/45
    
    Q = 1/3
    # Q = 1/SAMPLE_AMOUNT * np.matmul(X_2and3.T,X_2and3)
    
    
    dual_density = np.zeros((TIME_LEVELS, TIME_LEVELS, SAMPLE_AMOUNT))
    
    
    m = Model()
    
    pi = [[[m.add_var(lb=0,name="pi") for i in range(SAMPLE_AMOUNT)] for t2 in range(TIME_LEVELS)] for t2 in range(TIME_LEVELS)]
    
    for t1 in range(TIME_LEVELS):
        for t2 in range(TIME_LEVELS):
            
            m += xsum(pi[t1][t2][i] for i in range(SAMPLE_AMOUNT)) == SAMPLE_AMOUNT/TIME_LEVELS**2
            m += xsum(pi[t1][t2][i]*X1[i] for i in range(SAMPLE_AMOUNT)) == xsum(X1[i] for i in range(SAMPLE_AMOUNT))/TIME_LEVELS**2
        
    for i in range(SAMPLE_AMOUNT):
        m += xsum(xsum(pi[t1][t2][i] for t1 in range(TIME_LEVELS)) for t2 in range(TIME_LEVELS)) == 1
        
    m.objective = maximize(
        xsum(
            xsum(
                xsum(
                    (t1/TIME_LEVELS * Y1[i] + t2/TIME_LEVELS * Y2[i] * pi[t-1][i]) for i in range(SAMPLE_AMOUNT)
                ) for t2 in range(1,TIME_LEVELS+1)
            )for t1 in range(1,TIME_LEVELS+1)
        )
    )
    
    status = m.optimize(max_seconds=600)
    
    print(time.time()-start_time)
    if not status == OptimizationStatus.OPTIMAL:
        print("SOMETHING WRONG!")
        break

    else:
        t1=0
        t2=0
        i=0
        for v in m.vars: 
            dual_density[t1][t2][i] = v.x
            i+=1
            if i == SAMPLE_AMOUNT:
                i=0
                t2+=1
            if t2 == TIME_LEVELS:
                t2=0
                t1+=1
    
    # calculate target statistics
    
    b=0
    for t1 in range(1,TIME_LEVELS+1):
        for t2 in range(1,TIME_LEVELS+1):
            b +=  (t1/TIME_LEVELS)**2 * (dual_density[t1-1][t2-1])
        
        
    # S = 1/np.sqrt(SAMPLE_AMOUNT) * np.matmul(X_2and3.T,b)
    S = 1/np.sqrt(SAMPLE_AMOUNT) * np.dot(X2,b)
    
    T_STAT = S**2/(A*Q)
    
    collection[exp] = T_STAT
    
    print(T_STAT)
    print(time.time()-start_time)
    
    batch += 1
    
   
    if batch == 10:
        print(time.time()-start_time)
        print("experiment ",exp)
        batch = 0


ImportError: dlopen(/Users/turquoisekitty/opt/anaconda3/envs/working/lib/python3.8/site-packages/_cffi_backend.cpython-38-darwin.so, 2): Library not loaded: @rpath/libffi.7.dylib
  Referenced from: /Users/turquoisekitty/opt/anaconda3/envs/working/lib/python3.8/site-packages/_cffi_backend.cpython-38-darwin.so
  Reason: image not found